# Stacked LSTM

In [58]:
import numpy as np

from sklearn.preprocessing import MinMaxScaler

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

from bokeh.layouts import column
from bokeh.plotting import figure
from bokeh.io import output_notebook, show


np.random.seed(17)
output_notebook()

Loading BokehJS ...

### Design some artificial data

In [87]:
param = 20.
small_periods = 200
scale = 1.1
pick_freq = 10
dataset_1 = np.arange(param)
dataset_2 = np.arange(param)
dataset_3 = np.arange(param)

for i in range(1, small_periods):
    # Simple case
    d_1 = (np.arange(param) + (pick_freq * i  / 2)) * (scale + i / 50)
    dataset_1 = np.concatenate((dataset_1, d_1))
    # Interesting case
    d_2 = (np.arange(param) + (pick_freq * np.sin(i))) * (scale + i / 1)
    dataset_2 = np.concatenate((dataset_2, d_2))
    # Another interesting one
    d_3 = (np.arange(param) + (pick_freq * (i / 2) * np.sin(i))) * (scale + i / 100)
    dataset_3 = np.concatenate((dataset_3, d_3))


print(dataset_1.shape)
print(dataset_2.shape)
print(dataset_3.shape)

(4000,)
(4000,)
(4000,)


In [88]:
p_1 = figure(title='Dataset 1', plot_height=400, plot_width=900)
p_1.line(np.arange(len(dataset_1)), dataset_1, line_width=1)
p_2 = figure(title='Dataset 2', plot_height=400, plot_width=900)
p_2.line(np.arange(len(dataset_2)), dataset_2, line_width=1)
p_3 = figure(title='Dataset 3', plot_height=400, plot_width=900)
p_3.line(np.arange(len(dataset_3)), dataset_3, line_width=1)
show(column(p_1, p_2, p_3))

### Prepare data

In [89]:
scaler_1 = MinMaxScaler(feature_range=(0, 1))
scaler_2 = MinMaxScaler(feature_range=(0, 1))
scaler_3 = MinMaxScaler(feature_range=(0, 1))

ds_1 = scaler_1.fit_transform(dataset_1.reshape(-1, 1))
ds_2 = scaler_2.fit_transform(dataset_2.reshape(-1, 1))
ds_3 = scaler_3.fit_transform(dataset_3.reshape(-1, 1))

# scaler.inverse_transform(array) for inverse

In [92]:
p_1 = figure(title='Dataset 1 Scaled', plot_height=400, plot_width=900)
p_1.line(np.arange(len(ds_1)), ds_1.reshape(-1), line_width=1)
p_2 = figure(title='Dataset 2 Scaled', plot_height=400, plot_width=900)
p_2.line(np.arange(len(ds_2)), ds_2.reshape(-1), line_width=1)
p_3 = figure(title='Dataset 3 Scaled', plot_height=400, plot_width=900)
p_3.line(np.arange(len(ds_3)), ds_3.reshape(-1), line_width=1)
show(column(p_1, p_2, p_3))

In [95]:
def create_input(data, window_size=1):
    x, y = [], []
    for i in range(len(data) - window_size - 1):
        x.append(data[i:i + window_size, 0])
        y.append(data[i + window_size, 0])
    return np.array(x).reshape(data.shape[0], data.shape[1], 1), np.array(y)

def split_data(data, percentage=0.8):
    return data[:len(data) * percentage,:], data[len(data) * percentage:,:]

In [ ]:
train_1, test_1 = split_data(ds_1)
train_1_x, train_1_y = create_input(train_1)
test_1_x, test_1_y = create_input(test_1)

### Prepare models

In [ ]:
batch_size = 32
window_size = 1
timesteps = None